In [95]:
import feedparser
import sqlite3
import csv
import json

# Function to fetch news from an RSS Feed

In [96]:
def fetch_news(feed_url):
    feed = feedparser.parse(feed_url)
    articles = []

    for entry in feed.entries:
        article = {
            'title': entry.get('title',''),
            'summary': entry.get('summary',''),
            'published': entry.get('published',''),
            'category': None  # Placeholder for category
        }
        articles.append(article)

    return articles

# Function to Categorize new articles

In [97]:
def categorize_articles(articles):
    categorized_articles = {
        'Terriorism/ Protest/ Political Unrest/ Riot': [],
        'Positive/Uplifiting':[],
        'Natural Disasters':[],
        'Others':[]
    }
    
    for article in articles:
        # Implement your categorization logic based on keywords, topics, etc.
        # For simplicity, let's assume a basic keyword check for this example.
        title_lower = article['title'].lower()
        summary_lower = article['summary'].lower()
        
        if any(keyword in title_lower or keyword in summary_lower for keyword in ['terrorism', 'protest', 'unrest', 'riot']):
            article['category'] = 'Terrorism/Protest/Political Unrest/Riot'
        elif any(keyword in title_lower or keyword in summary_lower for keyword in ['positive', 'uplifting']):
            article['category'] = 'Positive/Uplifting'
        elif any(keyword in title_lower or keyword in summary_lower for keyword in ['natural disaster', 'natural calamity']):
            article['category'] = 'Natural Disasters'
        else:
            article['category'] = 'Others'

       # Ensure the category key exists in the dictionary
        if article['category'] not in categorized_articles:
            categorized_articles[article['category']] = []

        categorized_articles[article['category']].append(article)

    return categorized_articles

# Function to store article in SQLite database

In [98]:
def store_in_database(articles):
    conn = sqlite3.connect('news_database.db')
    cursor = conn.cursor()

    # Create a table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            summary TEXT,
            published TEXT,
            category TEXT
        )
    ''')

    # Insert articles into the database
    for article in articles:
        cursor.execute('''
            INSERT INTO news (title, summary, published, category)
            VALUES (?, ?, ?, ?)
        ''', (article['title'], article['summary'], article['published'], article['category']))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

In [99]:
# Function to export articles to CSV
def export_to_csv(articles, filename='news_articles.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['title', 'published', 'summary', 'category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for article in articles:
            writer.writerow(article)

# Function to export articles to JSON
def export_to_json(articles, filename='news_articles.json'):
    with open(filename, 'w', encoding='utf-8') as jsonfile:
        json.dump(articles, jsonfile, ensure_ascii=False, indent=2)

# Define RSS Feed URLs

In [100]:
rss_feeds = [
    'http://rss.cnn.com/rss/cnn_topstories.rss',
    'https://cms.qz.com/feed',
    'https://moxie.foxnews.com/google-publisher/politics.xml',
    'http://feeds.reuters.com/reuters/businessNews',
    'https://feeds.feedburner.com/NewshourWorld',
    'https://feeds.bbci.co.uk/news/world/asia/india/rss.xml'
]

# Fetching news articles from each feed

In [101]:
all_articles = []
for feed_url in rss_feeds:
    articles = fetch_news(feed_url)
    all_articles.extend(articles)

In [102]:
# Categorize articles
categorized_articles = categorize_articles(all_articles)

# Store articles in the database
store_in_database(all_articles)

OperationalError: table news has no column named published

In [103]:
# Display the categorized articles
for category, articles in categorized_articles.items():
    print(f"{category} Articles:")
    for i, article in enumerate(articles, start=1):
        print(f"Article {i}:")
        print(f"Title: {article['title']}")
        print(f"Published: {article['published']}")
        print(f"Summary: {article['summary']}")
        print("\n" + "="*50 + "\n")

Terriorism/ Protest/ Political Unrest/ Riot Articles:
Positive/Uplifiting Articles:
Natural Disasters Articles:
Others Articles:
Article 1:
Title: Some on-air claims about Dominion Voting Systems were false, Fox News acknowledges in statement after deal is announced
Published: Wed, 19 Apr 2023 12:44:51 GMT
Summary: 


Article 2:
Title: Dominion still has pending lawsuits against election deniers such as Rudy Giuliani and Sidney Powell
Published: 
Summary: 


Article 3:
Title: Here are the 20 specific Fox broadcasts and tweets Dominion says were defamatory
Published: Mon, 17 Apr 2023 16:01:11 GMT
Summary: • Fox-Dominion trial delay 'is not unusual,' judge says
• Fox News' defamation battle isn't stopping Trump's election lies


Article 4:
Title: Judge in Fox News-Dominion defamation trial: 'The parties have resolved their case'
Published: Wed, 19 Apr 2023 08:28:17 GMT
Summary: The judge just announced in court that a settlement has been reached in the historic defamation case between Fo

In [104]:
# Export articles to CSV
export_to_csv(all_articles)

# Export articles to JSON
export_to_json(all_articles)